In [1]:
!pip install ultralytics==8.0.196
!pip install roboflow

In [2]:
from roboflow import Roboflow
rf = Roboflow(api_key="")
project = rf.workspace("dsc-190-aks").project("shelf-picker")
version = project.version(7)
dataset = version.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Shelf-Picker-7 in yolov8:: 100%|██████████| 62098/62098 [00:01<00:00, 38750.16it/s]


In [4]:
# Train
!yolo task=detect \
mode=train \
model=yolov8s.pt \
data={dataset.location}/data.yaml \
epochs=500 \
imgsz=320 \
batch=-1

100%|██████████████████████████████████████| 21.5M/21.5M [00:00<00:00, 46.7MB/s]
New https://pypi.org/project/ultralytics/8.1.27 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.196 🚀 Python-3.10.13 torch-2.2.1 CUDA:0 (NVIDIA GeForce RTX 4090, 24217MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/home/Shelf-Picker-7/data.yaml, epochs=500, patience=50, batch=-1, imgsz=320, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_con

In [5]:
# Validate
!yolo task=detect \
mode=val \
model=runs/detect/train/weights/best.pt \
data={dataset.location}/data.yaml

Ultralytics YOLOv8.0.196 🚀 Python-3.10.13 torch-2.2.1 CUDA:0 (NVIDIA GeForce RTX 4090, 24217MiB)
Model summary (fused): 168 layers, 11126358 parameters, 0 gradients, 28.4 GFLOPs
val: Scanning /home/Shelf-Picker-7/valid/labels.cache... 632 images, 193 backgro
                 Class     Images  Instances      Box(P          R      mAP50  m
                   all        632        799      0.943      0.892       0.93      0.778
                person        632        765      0.899      0.784      0.864      0.622
                 robot        632         34      0.988          1      0.995      0.934
Speed: 0.1ms preprocess, 0.5ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to runs/detect/val
💡 Learn more at https://docs.ultralytics.com/modes/val


In [6]:
# Predict
!yolo task=detect \
mode=predict \
model=runs/detect/train/weights/best.pt \
conf=0.25 \
source={dataset.location}/test/images

Ultralytics YOLOv8.0.196 🚀 Python-3.10.13 torch-2.2.1 CUDA:0 (NVIDIA GeForce RTX 4090, 24217MiB)
Model summary (fused): 168 layers, 11126358 parameters, 0 gradients, 28.4 GFLOPs

image 1/311 /home/Shelf-Picker-7/test/images/-494-_JPG_jpg.rf.4eac6f601e233e50b219ab4c90a02951.jpg: 320x320 6 persons, 1.6ms
image 2/311 /home/Shelf-Picker-7/test/images/-499-_JPG_jpg.rf.054b834a591ea044870e083fb6f99126.jpg: 320x320 6 persons, 1.9ms
image 3/311 /home/Shelf-Picker-7/test/images/-5-_png_jpg.rf.5b81e02842b531478cd5418a85bfdb5e.jpg: 320x320 1 person, 1.6ms
image 4/311 /home/Shelf-Picker-7/test/images/-501-_JPG_jpg.rf.1d33c2b0ae3aa208217d5fe63b12cd4a.jpg: 320x320 7 persons, 1.5ms
image 5/311 /home/Shelf-Picker-7/test/images/-505-_JPG_jpg.rf.6894bdd3f7ea15621103360bb6e7b908.jpg: 320x320 1 person, 1.5ms
image 6/311 /home/Shelf-Picker-7/test/images/-517-_JPG_jpg.rf.7bea254ce3fbdf8f263121addb70c677.jpg: 320x320 1 person, 1.6ms
image 7/311 /home/Shelf-Picker-7/test/images/-534-_JPG_jpg.rf.3b31b943931f2c

In [ ]:
project.version(1).deploy(model_type="yolov8", model_path=f"runs/detect/train/")